# 체크용

In [2]:
import pandas as pd
import os

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
print(len(df_match))

fs=os.listdir('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/')
len(fs)

10156


8804

In [15]:
df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

nonlist=[]
yeslist=[]
srate= 100
for i in range(len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'+filename):
        nonlist.append(df_match.loc[i,'path'])
    else:
        yeslist.append(df_match.loc[i,'path'])
        
len(nonlist),len(yeslist)

10156


(1352, 8804)

In [16]:
import pickle

cid = pickle.load(open('../../DL_model/caseids_diff','rb'))
cid = [e[:-3]+'vital' for e in cid]

existed = []
still=[]
for p in nonlist:
    if p in cid:
        existed.append(p)
    if p in yeslist:
        still.append(p)
len(cid),len(existed),len(still)

(591, 190, 1132)

# ECG 250Hz

In [4]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
import vitaldb

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 250
for i in range(len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    if True:
    #if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename):
        print(i, end=' ')
        #try: 
        rec_path = '../../../cranberry2/Preprocessing/vital_data/pacu_ecg_pleth/pacu_ecg_pleth/'+df_match.loc[i,'path']
        #print(rec_path)
        #print('1')
        vfile_ecg = vitaldb.vital_recs_time(rec_path,['SNUADCW/ECG_II'],interval=0.004).flatten()

        #print(vfile)
        if len(vfile_ecg[0])==0 :
            print('index ',df_match.loc[i,'index'],' OPID ',df_match.loc[i,'opid'],'empty vital file')
        else:
            Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
            #print(vfile[1])
            dics = {'Abs_time':Abs_time,'ECG':vfile_ecg[0]}
            df_vital = pd.DataFrame(data=dics)
            end_idx_list = df_vital[df_vital['Abs_time'] == df_match.loc[i,'NRS_time']].index.tolist()
            print('len', len(end_idx_list))
            print(df_vital.loc[end_idx_list[0],'Abs_time'],'...',df_match.loc[i,''])
            if len(end_idx_list)==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")

            else:
                end_idx=end_idx_list[0]+1
                start_idx= end_idx - 5 * 60 * srate
                start_idx_n = max(0,start_idx)

                ext = df_vital[start_idx_n:end_idx]
                extr=ext.dropna(subset=['ECG'])
                extr=extr.drop_duplicates(subset=['ECG'])

                if len(extr)>1:
                    if start_idx <0 :
                        fit = (-1)*end_idx 
                        extraction = pd.DataFrame(index=range(5*60*srate),columns=['Abs_time','ECG'])
                        extraction[fit:]=ext

                    else:
                        extraction = ext
                    with open('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename,'wb') as f:
                        pickle.dump(extraction, f)
                    print('...done')
                else:
                    print('...결측치/직선 데이터')
        #except:
            #print('error: ',i,df_match.loc[i,'opid'],df_match.loc[i,'path'])

10156
85 ...결측치/직선 데이터
87 ...결측치/직선 데이터
94 ...결측치/직선 데이터
98 ...결측치/직선 데이터
103 ...결측치/직선 데이터
114 ...결측치/직선 데이터
116 ...결측치/직선 데이터
120 ...결측치/직선 데이터
370 ...결측치/직선 데이터
372 ...결측치/직선 데이터
395 ...결측치/직선 데이터
449 ...결측치/직선 데이터
594 ...결측치/직선 데이터
599 ...결측치/직선 데이터
602 ...결측치/직선 데이터
608 ...결측치/직선 데이터
662 ...결측치/직선 데이터
663 ...결측치/직선 데이터
786 ...결측치/직선 데이터
787 ...결측치/직선 데이터
1119 ...결측치/직선 데이터
1120 ...결측치/직선 데이터
1121 ...결측치/직선 데이터
1122 ...결측치/직선 데이터
1123 ...결측치/직선 데이터
1124 ...결측치/직선 데이터
1125 ...결측치/직선 데이터
1126 ...결측치/직선 데이터
1127 ...결측치/직선 데이터
1128 ...결측치/직선 데이터
1129 ...결측치/직선 데이터
1130 ...결측치/직선 데이터
1131 ...결측치/직선 데이터
1132 ...결측치/직선 데이터
1133 ...결측치/직선 데이터
1134 ...결측치/직선 데이터
1135 ...결측치/직선 데이터
1136 ...결측치/직선 데이터
1137 ...결측치/직선 데이터
1138 ...결측치/직선 데이터
1139 ...결측치/직선 데이터
1140 ...결측치/직선 데이터
1141 ...결측치/직선 데이터
1142 ...결측치/직선 데이터
1143 ...결측치/직선 데이터
1144 ...결측치/직선 데이터
1145 ...결측치/직선 데이터
1146 ...결측치/직선 데이터
1147 ...결측치/직선 데이터
1148 ...결측치/직선 데이터
1149 ...결측치/직선 데이터
1150 ...결측치/직선 데이터
1153 ...done
1156 ...done
1157

## 수정된 코드

In [32]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
import vitaldb

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''


error_list=[]
srate= 250
for i in range(len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    vital_path = '../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename
    
    if os.path.exists(vital_path):
    #if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename):
        print(i, end=' ')

        rec_path = '../../../cranberry2/Preprocessing/vital_data/pacu_full_vital/'+df_match.loc[i,'path']
        
        
        # vitaldb 에러에 해당되는 케이스
        df_vital = pickle.load(open(vital_path,'rb'))
        
        if np.sum(~np.isnan(df_vital['ECG'].tolist())) != 0:
            if np.nanmean(df_vital['ECG'].tolist()) > -4:
                print('already done')
                continue
        
        
        error_list.append(filename)
        print(df_match.loc[i,'path'], end='... ')
        
        vfile_ecg = vitaldb.vital_recs_time(rec_path,['SNUADCW/ECG_II'],interval=0.004).flatten()

        
        if len(vfile_ecg[0])==0 :
            print('index ',df_match.loc[i,'index'],' OPID ',df_match.loc[i,'opid'],'empty vital file')
            
        else:
            Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
            #print(vfile[1])
            dics = {'Abs_time':Abs_time,'ECG':vfile_ecg[0]}
            df_vital = pd.DataFrame(data=dics)
            end_idx_list = df_vital[(df_vital['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_vital['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

            
            if len(end_idx_list)==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")

            else:
                end_idx=end_idx_list[0]+1
                start_idx= end_idx - 5 * 60 * srate
                start_idx_n = max(0,start_idx)

                ext = df_vital[start_idx_n:end_idx]
                extr=ext.dropna(subset=['ECG'])
                extr=extr.drop_duplicates(subset=['ECG'])

                if len(extr)>1:
                    if start_idx <0 :
                        fit = (-1)*end_idx 
                        extraction = pd.DataFrame(index=range(5*60*srate),columns=['ECG'])
                        

                        col_ecg = np.full(5*60*srate, np.nan)
                        col_ecg[fit:] = ext['ECG'][fit:].tolist()                    
                        extraction['ECG'] = col_ecg                        
                        

                    else:
                        extraction = ext
                        
                    with open('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename,'wb') as f:
                        pickle.dump(extraction, f)
                    print('...done')
                else:
                    print('...결측치/직선 데이터')
                
        #except:
            #print('error: ',i,df_match.loc[i,'opid'],df_match.loc[i,'path'])

10156
0 already done
1 already done
2 already done
3 already done
4 already done
5 already done
6 already done
7 already done
8 already done
9 already done
10 already done
11 already done
12 already done
13 already done
14 already done
15 already done
16 already done
17 already done
18 already done
19 already done
20 already done
21 already done
22 already done
23 already done
24 already done
25 already done
26 already done
27 already done
28 already done
29 already done
30 already done
31 already done
32 already done
33 already done
34 already done
35 already done
36 already done
37 PACU1_3_190412_163450.vital... ...done
38 already done
39 already done
40 already done
41 already done
42 already done
43 already done
44 already done
45 PACU1_3_190413_012948.vital... ...done
46 already done
47 PACU1_3_190413_065925.vital... ...done
48 already done
49 already done
50 already done
51 already done
52 already done
53 already done
54 already done
55 already done
56 already done
57 already don

447 already done
448 already done
450 already done
451 already done
452 already done
453 already done
454 already done
455 already done
456 already done
457 already done
458 already done
459 already done
460 already done
461 already done
462 already done
463 already done
464 already done
465 already done
466 already done
467 already done
468 already done
469 PACU1_9_190515_160239.vital... ...done
470 already done
471 already done
472 already done
473 already done
474 already done
475 already done
476 already done
477 already done
478 already done
479 already done
480 PACU1_3_190515_190346.vital... ...done
481 already done
482 PACU1_1_190515_202235.vital... ...done
483 already done
484 already done
485 already done
486 already done
487 already done
488 already done
489 PACU1_1_190516_090804.vital... ...done
490 already done
491 already done
492 already done
493 already done
494 already done
495 already done
496 already done
497 already done
498 already done
499 already done
500 already 

880 PACU1_5_190617_125936.vital... ...done
881 already done
882 PACU1_5_190617_164932.vital... ...done
883 already done
884 already done
885 already done
886 already done
887 PACU1_5_190617_184947.vital... ...done
888 already done
889 already done
890 already done
891 PACU1_8_190703_100905.vital... ...done
892 already done
893 already done
894 already done
895 already done
896 already done
897 already done
898 already done
899 already done
900 already done
901 already done
902 already done
903 already done
904 already done
905 already done
906 already done
907 already done
908 already done
909 already done
910 already done
911 already done
912 already done
913 already done
914 already done
915 already done
916 already done
917 already done
918 already done
919 already done
920 already done
921 already done
922 PACU1_2_190703_202359.vital... ...done
923 already done
924 already done
925 already done
926 already done
927 already done
928 already done
929 already done
930 already done
931

1362 already done
1363 already done
1364 already done
1365 already done
1366 already done
1367 already done
1368 already done
1369 already done
1370 already done
1371 already done
1372 already done
1373 already done
1374 already done
1375 already done
1376 already done
1377 already done
1378 already done
1379 already done
1380 already done
1381 already done
1382 already done
1383 already done
1384 already done
1385 already done
1386 already done
1387 already done
1388 already done
1389 already done
1390 already done
1391 already done
1392 already done
1393 already done
1394 already done
1395 already done
1396 already done
1397 already done
1398 already done
1399 already done
1400 already done
1401 already done
1402 already done
1403 already done
1404 already done
1405 already done
1406 already done
1407 already done
1408 already done
1409 already done
1410 already done
1411 already done
1412 already done
1413 already done
1414 already done
1415 already done
1416 already done
1417 alrea

1827 PACU1_1_190823_200839.vital... ...done
1828 already done
1829 already done
1830 already done
1831 already done
1832 already done
1833 already done
1834 already done
1835 already done
1836 already done
1837 already done
1838 already done
1839 already done
1840 already done
1841 already done
1842 already done
1843 already done
1844 already done
1845 already done
1846 already done
1847 already done
1848 already done
1849 already done
1850 already done
1851 already done
1852 already done
1853 already done
1854 PACU1_11_190827_131853.vital... ...done
1855 already done
1856 already done
1857 already done
1858 already done
1859 already done
1860 already done
1861 already done
1862 PACU1_5_190827_163756.vital... ...done
1863 already done
1864 already done
1865 already done
1866 already done
1867 already done
1868 already done
1869 already done
1870 already done
1871 already done
1872 PACU1_3_190827_194409.vital... ...done
1873 already done
1874 already done
1875 already done
1876 already 

...

8233 already done
8234 already done
8235 already done
8236 already done
8237 already done
8238 already done
8239 already done
8240 already done
8241 already done
8242 already done
8243 already done
8244 already done
8245 already done
8246 already done
8247 already done
8248 already done
8249 already done
8250 already done
8251 already done
8252 already done
8253 already done
8254 already done
8255 already done
8256 already done
8257 already done
8258 already done
8259 already done
8260 already done
8261 already done
8262 already done
8263 already done
8264 already done
8265 already done
8266 already done
8267 already done
8268 already done
8269 already done
8270 already done
8271 already done
8272 already done
8273 already done
8274 already done
8275 already done
8276 already done
8277 already done
8278 already done
8279 already done
8280 already done
8281 already done
8282 already done
8283 already done
8284 already done
8285 already done
8286 already done
8287 already done
8288 alrea

8696 already done
8697 already done
8698 already done
8699 already done
8700 already done
8701 already done
8702 already done
8703 already done
8704 already done
8705 already done
8706 already done
8707 already done
8708 already done
8709 already done
8710 already done
8711 already done
8712 already done
8713 already done
8714 already done
8715 already done
8716 already done
8717 already done
8718 already done
8719 already done
8720 already done
8721 already done
8722 already done
8723 already done
8724 already done
8725 already done
8726 already done
8727 already done
8728 already done
8729 already done
8730 already done
8731 already done
8732 already done
8733 already done
8734 already done
8735 already done
8736 already done
8737 already done
8738 already done
8739 already done
8740 already done
8741 already done
8742 already done
8743 already done
8744 already done
8745 already done
8746 already done
8747 already done
8748 already done
8749 already done
8750 already done
8751 alrea

9158 already done
9159 already done
9160 already done
9161 already done
9162 already done
9163 already done
9164 already done
9165 already done
9166 already done
9167 already done
9168 already done
9169 already done
9170 already done
9171 already done
9172 already done
9173 already done
9174 already done
9175 already done
9176 already done
9177 already done
9178 already done
9179 already done
9180 already done
9181 already done
9182 already done
9183 already done
9184 already done
9185 already done
9186 already done
9187 already done
9188 already done
9189 already done
9190 already done
9191 already done
9192 already done
9193 already done
9194 already done
9195 already done
9196 already done
9197 already done
9198 already done
9199 already done
9200 already done
9201 already done
9202 already done
9203 already done
9204 already done
9205 already done
9206 already done
9207 already done
9208 already done
9209 already done
9210 already done
9211 already done
9212 already done
9213 alrea

9614 already done
9615 already done
9616 already done
9617 already done
9618 already done
9619 already done
9620 already done
9621 already done
9622 already done
9623 already done
9624 already done
9625 already done
9626 already done
9627 already done
9628 already done
9629 already done
9630 already done
9631 already done
9632 already done
9633 already done
9634 already done
9635 already done
9636 already done
9637 already done
9638 already done
9639 already done
9640 already done
9641 already done
9642 already done
9643 already done
9644 already done
9645 already done
9646 already done
9647 already done
9648 already done
9649 already done
9650 already done
9651 already done
9652 already done
9653 already done
9654 already done
9655 already done
9656 already done
9657 already done
9658 already done
9659 already done
9660 already done
9661 already done
9662 already done
9663 already done
9664 already done
9665 already done
9666 already done
9667 already done
9668 already done
9669 alrea

10068 already done
10069 already done
10070 already done
10071 already done
10072 already done
10073 already done
10074 already done
10075 already done
10076 already done
10077 already done
10078 already done
10079 already done
10080 already done
10081 already done
10082 already done
10083 already done
10084 already done
10085 already done
10086 already done
10087 already done
10088 already done
10089 already done
10090 already done
10091 already done
10092 already done
10093 already done
10094 already done
10095 already done
10096 already done
10097 already done
10098 already done
10099 already done
10100 already done
10101 already done
10102 already done
10103 already done
10104 already done
10105 already done
10106 already done
10107 already done
10108 already done
10109 already done
10110 already done
10111 already done
10112 already done
10113 already done
10114 already done
10115 already done
10116 already done
10117 already done
10118 already done
10119 already done
10120 alread

In [33]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
import vitaldb

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''


err_list=[]
srate= 250
for i in range(len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    vital_path = '../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename
    
    if os.path.exists(vital_path):
    #if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename):
        print(i, end=' ')

        rec_path = '../../../cranberry2/Preprocessing/vital_data/pacu_full_vital/'+df_match.loc[i,'path']
        
        
        # vitaldb 에러에 해당되는 케이스
        df_vital = pickle.load(open(vital_path,'rb'))
        
        if np.sum(~np.isnan(df_vital['ECG'].tolist())) != 0:
            if np.nanmean(df_vital['ECG'].tolist()) > -4:
                print('already done')
                continue
        else:
            err_list.append(filename)

10156
0 already done
1 already done
2 already done
3 already done
4 already done
5 already done
6 already done
7 already done
8 already done
9 already done
10 already done
11 already done
12 already done
13 already done
14 already done
15 already done
16 already done
17 already done
18 already done
19 already done
20 already done
21 already done
22 already done
23 already done
24 already done
25 already done
26 already done
27 already done
28 already done
29 already done
30 already done
31 already done
32 already done
33 already done
34 already done
35 already done
36 already done
37 already done
38 already done
39 already done
40 already done
41 already done
42 already done
43 already done
44 already done
45 already done
46 already done
47 already done
48 already done
49 already done
50 already done
51 already done
52 already done
53 already done
54 already done
55 already done
56 already done
57 already done
58 already done
59 already done
60 already done
61 already done
62 already d

503 already done
504 already done
505 already done
506 already done
507 already done
508 already done
509 already done
510 already done
511 already done
512 already done
513 already done
514 already done
515 already done
516 already done
517 already done
518 already done
519 already done
520 already done
521 already done
522 already done
523 already done
524 already done
525 already done
526 already done
527 already done
528 already done
529 already done
530 already done
531 already done
532 already done
533 already done
534 already done
535 already done
536 already done
537 already done
538 already done
539 already done
540 already done
541 already done
542 already done
543 already done
544 already done
545 already done
546 already done
547 already done
548 already done
549 already done
550 already done
551 already done
552 already done
553 already done
554 already done
555 already done
556 already done
557 already done
558 already done
559 already done
560 already done
561 already do

996 already done
997 already done
998 already done
999 already done
1000 already done
1001 already done
1002 already done
1003 already done
1004 already done
1005 already done
1006 already done
1007 already done
1008 already done
1009 already done
1010 already done
1011 already done
1012 already done
1013 already done
1014 already done
1015 already done
1016 already done
1017 already done
1018 already done
1019 already done
1020 already done
1021 already done
1022 already done
1023 already done
1024 already done
1025 already done
1026 already done
1027 already done
1028 already done
1029 already done
1030 already done
1031 already done
1032 already done
1033 already done
1034 already done
1035 already done
1036 already done
1037 already done
1038 already done
1039 already done
1040 already done
1041 already done
1042 already done
1043 already done
1044 already done
1045 already done
1046 already done
1047 already done
1048 already done
1049 already done
1050 already done
1051 already d

1491 already done
1492 already done
1493 already done
1494 already done
1495 already done
1496 already done
1497 already done
1498 already done
1499 already done
1500 already done
1501 already done
1502 already done
1503 already done
1504 already done
1505 already done
1506 already done
1507 already done
1508 already done
1509 already done
1510 already done
1511 already done
1512 already done
1513 already done
1514 already done
1515 already done
1516 already done
1517 already done
1518 already done
1519 already done
1520 already done
1521 already done
1522 already done
1523 already done
1524 already done
1525 already done
1526 already done
1527 already done
1528 already done
1529 already done
1530 already done
1531 already done
1532 already done
1533 already done
1534 already done
1535 already done
1536 already done
1537 already done
1538 already done
1539 already done
1540 already done
1541 already done
1542 already done
1543 already done
1544 already done
1545 already done
1546 alrea

1949 already done
1950 already done
1951 already done
1952 already done
1953 already done
1954 already done
1955 already done
1956 already done
1957 already done
1958 already done
1959 already done
1960 already done
1961 already done
1962 already done
1963 already done
1964 already done
1965 already done
1966 already done
1967 already done
1968 already done
1969 already done
1970 already done
1971 already done
1972 already done
1973 already done
1974 already done
1975 already done
1976 already done
1977 already done
1978 already done
1979 already done
1980 already done
1981 already done
1982 already done
1983 already done
1984 already done
1985 already done
1986 already done
1987 already done
1988 already done
1989 already done
1990 already done
1991 already done
1992 already done
1993 already done
1994 already done
1995 already done
1996 already done
1997 already done
1998 already done
1999 already done
2000 already done
2001 already done
2002 already done
2003 already done
2004 alrea

...

7935 already done
7936 already done
7937 already done
7938 already done
7939 already done
7940 already done
7941 already done
7942 already done
7943 already done
7944 already done
7945 already done
7946 already done
7947 already done
7948 already done
7949 already done
7950 already done
7951 already done
7952 already done
7953 already done
7954 already done
7955 already done
7956 already done
7957 already done
7958 already done
7959 already done
7960 already done
7961 already done
7962 already done
7963 already done
7964 already done
7965 already done
7966 already done
7967 already done
7968 already done
7969 already done
7970 already done
7971 already done
7972 already done
7973 already done
7974 already done
7977 already done
7978 already done
7979 already done
7980 already done
7981 already done
7982 already done
7983 already done
7984 already done
7985 already done
7986 already done
7987 already done
7988 already done
7989 already done
7990 already done
7991 already done
7992 alrea

8401 already done
8402 already done
8403 already done
8404 already done
8405 already done
8406 already done
8407 already done
8408 already done
8409 already done
8410 already done
8411 already done
8412 already done
8413 already done
8414 already done
8415 already done
8416 already done
8417 already done
8418 already done
8419 already done
8420 already done
8421 already done
8422 already done
8423 already done
8424 already done
8425 already done
8426 already done
8427 already done
8428 already done
8429 already done
8430 already done
8431 already done
8432 already done
8433 already done
8434 already done
8435 already done
8436 already done
8437 already done
8438 already done
8439 already done
8440 already done
8441 already done
8442 already done
8443 already done
8444 already done
8445 already done
8446 already done
8447 already done
8448 already done
8449 already done
8450 already done
8451 already done
8452 already done
8453 already done
8454 already done
8455 already done
8456 alrea

8869 already done
8870 already done
8871 already done
8872 already done
8873 already done
8874 already done
8875 already done
8876 already done
8877 already done
8878 already done
8879 already done
8880 already done
8881 already done
8882 already done
8883 already done
8884 already done
8885 already done
8886 already done
8887 already done
8888 already done
8889 already done
8890 already done
8891 already done
8892 already done
8893 already done
8894 already done
8895 already done
8896 already done
8897 already done
8898 already done
8899 already done
8900 already done
8901 already done
8902 already done
8903 already done
8904 already done
8905 already done
8906 already done
8907 already done
8908 already done
8909 already done
8910 already done
8911 already done
8912 already done
8913 already done
8914 already done
8915 already done
8916 already done
8917 already done
8918 already done
8919 already done
8920 already done
8921 already done
8922 already done
8923 already done
8924 alrea

9326 already done
9327 already done
9328 already done
9329 already done
9330 already done
9331 already done
9332 already done
9333 already done
9334 already done
9335 already done
9336 already done
9337 already done
9338 already done
9339 already done
9340 already done
9341 already done
9342 already done
9343 already done
9344 already done
9345 already done
9346 already done
9347 already done
9348 already done
9349 already done
9350 already done
9351 already done
9352 already done
9353 already done
9354 already done
9355 already done
9356 already done
9357 already done
9358 already done
9359 already done
9360 already done
9361 already done
9362 already done
9363 already done
9364 already done
9365 already done
9366 already done
9367 already done
9368 already done
9369 already done
9370 already done
9371 already done
9372 already done
9373 already done
9374 already done
9375 already done
9376 already done
9377 already done
9378 already done
9379 already done
9380 already done
9381 alrea

9781 already done
9782 already done
9783 already done
9784 already done
9785 already done
9786 already done
9787 already done
9788 already done
9789 already done
9790 already done
9791 already done
9792 already done
9793 already done
9794 already done
9795 already done
9796 already done
9797 already done
9798 already done
9799 already done
9800 already done
9801 already done
9802 already done
9803 already done
9804 already done
9805 already done
9806 already done
9807 already done
9808 already done
9809 already done
9810 already done
9811 already done
9812 already done
9813 already done
9814 already done
9815 already done
9816 already done
9817 already done
9818 already done
9819 already done
9820 already done
9821 already done
9822 already done
9823 already done
9824 already done
9825 already done
9826 already done
9827 already done
9828 already done
9829 already done
9830 already done
9831 already done
9832 already done
9833 already done
9835 already done
9836 already done
9837 alrea

In [34]:
err_list

[]

# NIBP PACU pickle로 저장하기

In [4]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
from pyvital2 import vitaldb

df_match = pd.read_csv('match_drug_arr_exclusion_20201110.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_IDpath = pd.read_csv('opid_fileid.csv')
#df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()]

for j in range(len(df_IDpath)):
    #try: 
    #opid match
    idx_list = df_match[df_match['opid']==df_IDpath.loc[j,'opid']].index.tolist()
    i = min(idx_list)

    rec_path = 'preop_vital/preop_vital/' + df_IDpath.loc[j,'fileid']
    df_IDpath.loc[j, 'file_path'] = df_match.loc[i, 'path']
    df_IDpath.loc[j, 'NRS'] = df_match.loc[i, 'Value']
    df_IDpath.loc[j, 'NRS_time'] = df_match.loc[i, 'NRS_time']
print(df_IDpath)
print(df_match)

           opid                  fileid                    file_path   NRS  \
0     190408044  A2_190408_103443.vital  PACU1_1_190408_115004.vital   6.0   
1     190408106  E3_190408_180036.vital  PACU1_1_190408_202459.vital   7.0   
2     190409012  A2_190409_073654.vital  PACU1_2_190409_110712.vital   4.0   
3     190409044  D5_190409_101311.vital  PACU1_3_190409_114333.vital   6.0   
4     190409057  F4_190409_105106.vital  PACU1_1_190409_133948.vital   8.0   
...         ...                     ...                          ...   ...   
4109  201030106  R9_201030_145209.vital  PACU1_1_201030_161119.vital   6.0   
4110  201030057  E6_201030_131959.vital  PACU1_3_201030_195639.vital   7.0   
4111  201030082  R2_201030_185910.vital  PACU1_5_201030_203752.vital   6.0   
4112  201030023  C4_201030_185047.vital  PACU1_1_201030_211816.vital   6.0   
4113  201031004  R9_201031_084856.vital  PACU1_3_201031_131135.vital  10.0   

                NRS_time  
0    2019-04-08 12:00:00  
1    2019

In [ ]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
from pyvital2 import vitaldb

'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 250
for i in range(len(df_match)):
    bedno = df_match.loc[i,'path'].split('_')[1]
    if int(bedno) <4:
        filename = str(df_match.loc[i,'Value'])+','+str(i)+','+ df_match.loc[i,'path']
        if not os.path.exists('vital_data/NIBP_250Hz_pacu_2min_pickle/'+filename):
            rec_path = 'PACU_vital/PACU1_1-3/'+df_match.loc[i,'path']
            vfile = vitaldb.vital_recs_time(rec_path,['Solar8000/NIBP_MBP'],interval=0.004).flatten()
            #print(vfile)
            if len(vfile[0])==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid'],'empty NIBP')
            else:
                Abs_time = [vfile[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile[0]))]
                #print(vfile[1])
                dics = {'Abs_time':Abs_time,'NIBP':vfile[0]}
                df_NIBP = pd.DataFrame(data=dics)
                end_idx_list = df_NIBP[(df_NIBP['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_NIBP['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

                if len(end_idx_list)==0:
                    print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")                  
                else:
                    end_idx=end_idx_list[0]
                    start_idx= max(0,end_idx - 2 * 60 * srate)
                    extraction = df_NIBP.loc[start_idx:end_idx]
                    with open('vital_data/NIBP_250Hz_pacu_2min_pickle/'+filename,'wb') as f:
                        pickle.dump(extraction, f)
                        
#이건 2분보다 짧아도 저장함

# NIBP PACU pickle로 저장하기

In [ ]:
import pandas as pd
import pickle
import datetime
import time
from pyvital2 import vitaldb
import numpy as np
import os

'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 250
for i in range(len(df_IDpath)):
    filename = df_IDpath.loc[i,'file_path']
    if not os.path.exists('vital_data/NIBP_250Hz_preop_2min_pickle/'+filename):
        rec_path = 'preop_vital/preop_vital/'+ df_IDpath.loc[i,'fileid']
        vfile = vitaldb.vital_recs(rec_path,['Solar8000/NIBP_MBP'],interval=0.004).flatten()[0:2*60*250]

        #print(vfile)
        if len(vfile) ==0 :
            print('index ',i,' OPID ',df_IDpath.loc[i,'opid'],'empty NIBP')
        else:
            dics = {'NIBP':vfile}
            df_NIBP = pd.DataFrame(data=dics)

            extraction = df_NIBP
            with open('vital_data/NIBP_250Hz_preop_2min_pickle/'+filename,'wb') as f:
                pickle.dump(extraction, f)
            print(i,'/',len(df_IDpath),' save')